# Exploration Notebook v0 - Jacopo

To avoid cluttering the main notebook, I'm going to do some exploration here, it might get very messy as a whole but it should be easy to follow.

This will probably be the first (zero) version of several notebooks. The version names should help with keeping track and I will name them -Jacopo to avoid confusion with the main notebook.

## Folder Structure
- `./`: Notebooks, Readme, gitignore and Data are in the root folder
- `data/ `: all data in this folder, will be ignored by git but it will eventually include quite a lot, subfolders like `out/` for results, `data-ssd/` for YouNiverse dataset symlinked to an ssd



In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%reload_ext autoreload

In [4]:
data_path = "data/data-ssd/"

# Load data
# 10k rows for each, sampled randomly
train = pd.read_csv(data_path + "train.csv", nrows=10000, )

NameError: name 'df_' is not defined